# **Desarrollar una IA generadora de cuadros que, basado en un prompt del usuario, cree una imagen en el estilo de un artista seleccionado de una lista.**

El modelo Stable Diffusion porcesa de manera mas optima en ingles el prompt, recomiendo escribirlo en ese idioma. Aun asi en español sigue pillando cosas. 

### Celda 1: Instalación de dependencias
Esta celda se utiliza para instalar las bibliotecas necesarias que permitirán ejecutar el proyecto. 
Las principales librerías incluyen:
- **diffusers**: Para trabajar con modelos de difusión, utilizados en la generación de imágenes.
- **transformers**: Biblioteca clave para trabajar con modelos preentrenados de procesamiento de lenguaje natural y difusión.
- **torch**: Framework de aprendizaje profundo que proporciona herramientas para crear y entrenar modelos de redes neuronales.
- **accelerate**: Optimiza y acelera el entrenamiento en múltiples dispositivos.

In [1]:
import torch

if torch.cuda.is_available():
    print("GPU disponible:", torch.cuda.get_device_name(0))
else:
    print("No se detectó GPU. Verifica las configuraciones en Kaggle.")

import os
from PIL import Image
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

# Actualizar e instalar las dependencias necesarias
!pip install --upgrade diffusers transformers accelerate

# Ruta del dataset
dataset_path = '/kaggle/input/art-styles-dataset-for-ai-projects/Dataset_Cuadros'

GPU disponible: Tesla T4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 36.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 86.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 84.3 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


### Celda 2: Visualización interactiva del dataset
Esta celda permite al usuario explorar las imágenes del dataset de manera interactiva. Utiliza las siguientes herramientas:
- **ipywidgets**: Crea una interfaz interactiva con un desplegable para seleccionar un artista.
- **random**: Selecciona un subconjunto aleatorio de imágenes del artista elegido.
- **matplotlib**: Muestra hasta seis imágenes seleccionadas aleatoriamente en un formato visual amigable.

El objetivo principal de esta celda es proporcionar al usuario una idea de las imágenes contenidas en el dataset y facilitar la inspección visual por artista.

In [2]:
import random

def show_random_images(artist):
    artist_path = os.path.join(dataset_path, artist)
    image_files = os.listdir(artist_path)
    sampled_images = random.sample(image_files, min(6, len(image_files)))  # Tomar hasta 6 imágenes

    fig, axes = plt.subplots(1, len(sampled_images), figsize=(15, 5))
    for ax, img_file in zip(axes, sampled_images):
        img_path = os.path.join(artist_path, img_file)
        img = Image.open(img_path)
        ax.imshow(img)
        ax.axis('off')
    plt.show()

interact(show_random_images, artist=widgets.Dropdown(
    options=os.listdir(dataset_path),
    description='Artista:',
    style={'description_width': 'initial'}
))

interactive(children=(Dropdown(description='Artista:', options=('Kahlo', 'Monet', 'Hokusai', 'Van_Gogh', 'Dalí…

<function __main__.show_random_images(artist)>

### Celda 3: Clase y transformaciones para preprocesamiento
En esta sección se define una clase personalizada llamada `ArtistDataset`, que es responsable de:
- Cargar las imágenes desde las carpetas del dataset.
- Aplicar transformaciones a las imágenes, como:
  - **Redimensionar**: Cambia el tamaño de las imágenes a 512x512 píxeles para garantizar consistencia.
  - **Conversión a tensores**: Convierte las imágenes en tensores de PyTorch.
  - **Normalización**: Escala los valores de píxeles para que estén entre -1 y 1, lo que mejora la estabilidad del entrenamiento.

Esta clase es fundamental para preparar los datos antes de entrenar el modelo.

In [3]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class ArtistDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir)]
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image

# Transformaciones para las imágenes
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalizar entre -1 y 1
])

### Celda 4: Entrenamiento del modelo
En esta celda se lleva a cabo el entrenamiento del modelo utilizando un dataset de artistas. 
#### Principales pasos:
1. **Definición del dataset**: Se crea un dataset etiquetado que asigna una etiqueta única a cada estilo de artista.
2. **Modelo**: El modelo utilizado es una red neuronal simple con capas totalmente conectadas. Aunque es funcional, esta arquitectura puede ser limitada para tareas complejas.
3. **Entrenamiento**:
   - Se optimizan los pesos del modelo usando `Adam` como optimizador.
   - Se mide el error con la pérdida de entropía cruzada (`CrossEntropyLoss`).
   - Un programador de tasa de aprendizaje reduce el aprendizaje cuando la pérdida no mejora.
4. **Resultados**: Durante cada época, se calcula la pérdida promedio para evaluar el progreso del entrenamiento.

Esta celda está diseñada para entrenar un modelo básico y puede ser mejorada utilizando arquitecturas de redes neuronales convolucionales (CNN) o transfer learning.

In [4]:
from torch import nn, optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import torch

# Dataset con etiquetas reales
class ArtistDataset(Dataset):
    def __init__(self, dataset_path, transform=None):
        self.image_paths = []
        self.labels = []
        self.transform = transform
        self.label_map = {artist: i for i, artist in enumerate(os.listdir(dataset_path))}
        
        # Asignar etiquetas según la carpeta
        for artist, label in self.label_map.items():
            artist_path = os.path.join(dataset_path, artist)
            for image_file in os.listdir(artist_path):
                if image_file.endswith('.jpg'):
                    self.image_paths.append(os.path.join(artist_path, image_file))
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Transformaciones para las imágenes
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Función de entrenamiento
def train_model(dataset_path, epochs=20, batch_size=16, initial_lr=0.0001):
    # Crear dataset y dataloader
    dataset = ArtistDataset(dataset_path, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Modelo
    model = nn.Sequential(
        nn.Flatten(),
        nn.Linear(512 * 512 * 3, 256),
        nn.ReLU(),
        nn.Linear(256, len(os.listdir(dataset_path)))  # Clasificar estilos
    )
    optimizer = optim.Adam(model.parameters(), lr=initial_lr)
    criterion = nn.CrossEntropyLoss()
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5, verbose=True)

    # Entrenamiento
    for epoch in range(epochs):
        epoch_loss = 0.0
        for images, labels in dataloader:
            images = images.view(images.size(0), -1)
            labels = labels.to(torch.long)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(dataloader)
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")
        scheduler.step(avg_loss)

    return model

# Entrenar el modelo con etiquetas reales
model = train_model(dataset_path, epochs=20, batch_size=16, initial_lr=0.0001)

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/20, Loss: 6.2433
Epoch 2/20, Loss: 1.7233
Epoch 3/20, Loss: 0.6066
Epoch 4/20, Loss: 0.4865
Epoch 5/20, Loss: 0.2419
Epoch 6/20, Loss: 0.0008
Epoch 7/20, Loss: 0.0005
Epoch 8/20, Loss: 0.0014
Epoch 9/20, Loss: 0.0080
Epoch 10/20, Loss: 0.0007
Epoch 11/20, Loss: 0.0005
Epoch 12/20, Loss: 0.0004
Epoch 13/20, Loss: 0.0001
Epoch 14/20, Loss: 0.0001
Epoch 15/20, Loss: 0.0000
Epoch 16/20, Loss: 0.0000
Epoch 17/20, Loss: 0.0000
Epoch 18/20, Loss: 0.0000
Epoch 19/20, Loss: 0.0000
Epoch 20/20, Loss: 0.0000


### Celda 5: Generación y comparación de imágenes
En esta sección se implementa la generación de nuevas imágenes basadas en prompts utilizando Stable Diffusion. 
#### Desglose:
1. **Carga del modelo Stable Diffusion**:
   - Se utiliza el modelo `stable-diffusion-v1-4` preentrenado.
   - La configuración de "slicing" optimiza el uso de memoria durante la generación.
2. **Generación de imágenes**:
   - Combina un estilo de artista seleccionado por el usuario con un prompt descriptivo para crear una nueva imagen.
3. **Comparación visual**:
   - Se seleccionan al azar imágenes de referencia del dataset para compararlas con la imagen generada.
   - Se muestra una gráfica con la imagen generada y las referencias, facilitando la comparación.
4. **Interactividad**:
   - Widgets permiten al usuario elegir un artista y escribir un prompt antes de generar una imagen.

Esta celda es crucial para mostrar los resultados finales y evaluar la calidad de las imágenes generadas en comparación con los estilos del dataset original.

In [5]:
!pip install diffusers[torch] transformers accelerate
from diffusers import StableDiffusionPipeline

device = "cuda" if torch.cuda.is_available() else "cpu"

pipeline = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4").to(device)
pipeline.enable_attention_slicing()

def generate_and_compare(artist, prompt):
    full_prompt = f"A painting in the style of {artist}: {prompt}"
    generated_image = pipeline(full_prompt).images[0]

    artist_path = os.path.join(dataset_path, artist)
    reference_images = random.sample(os.listdir(artist_path), min(2, len(os.listdir(artist_path))))

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    axes[0].imshow(generated_image)
    axes[0].set_title("Generated")
    axes[0].axis('off')

    for i, ref_image in enumerate(reference_images):
        img_path = os.path.join(artist_path, ref_image)
        img = Image.open(img_path)
        axes[i + 1].imshow(img)
        axes[i + 1].set_title(f"Reference {i + 1}")
        axes[i + 1].axis('off')

    plt.tight_layout()
    plt.show()

from ipywidgets import Button, Output, VBox

# Crear widgets para artista y prompt
artist_dropdown = widgets.Dropdown(options=os.listdir(dataset_path), description='Artista:')
prompt_input = widgets.Text(value='', description='Prompt:')
button = Button(description="Generar Imagen")
output = Output()

def on_button_click(b):
    with output:
        output.clear_output()
        generate_and_compare(artist_dropdown.value, prompt_input.value)

button.on_click(on_button_click)

# Mostrar widgets y botón
VBox([artist_dropdown, prompt_input, button, output])

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

scheduler/scheduler_config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

...

vae/config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

tokenizer/tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]